# Matrix Exponential Problem: Python vs NumPy

## Problem Description

We want to compute the **matrix exponential applied to a vector**:

$$
y = \exp(A) \cdot v
$$

where:

- $A$ is an $n \times n$ matrix (real or complex),  
- $v$ is an $n$-dimensional vector,  
- $\exp(A)$ is the **matrix exponential**, defined as:

$$
\exp(A) = I + A + \frac{A^2}{2!} + \frac{A^3}{3!} + \dots
$$

This operation is widely used in:

- Solving systems of **linear differential equations**,
- Physics and engineering simulations (e.g., quantum mechanics, control systems),
- Machine learning algorithms (e.g., continuous-time models),
- Any scenario requiring **linear transformations over time**.


## 1️⃣ Pure Python Implementation (Taylor Series)

This section computes y = exp(A)·v using **manual Python loops**.

In [1]:
import random
import time
import math
import numpy as np
n = 150
A = [[random.uniform(-1,1) for _ in range(n)] for _ in range(n)]
v = [random.uniform(-1,1) for _ in range(n)]

In [2]:
import time

# Assuming A and v are already defined
def matvec(A, v):
    n = len(A)
    result = []
    for i in range(n):
        total = 0
        for j in range(n):
            total += A[i][j] * v[j]
        result.append(total)
    return result


def matmul(A, B):
    n = len(A)
    result = []
    for i in range(n):
        row = []
        for j in range(n):
            value = 0
            for k in range(n):
                value += A[i][k] * B[k][j]
            row.append(value)
        result.append(row)
    return result


t0 = time.time()

K = 20
y_python = v[:]                       # Initialize y with v
Ak = [row[:] for row in A]            # Copy of A
fact = 1.0

for k in range(1, K + 1):
    fact *= k
    term = matvec(Ak, v)
    y_python = [y_python[i] + term[i] / fact for i in range(len(v))]
    Ak = matmul(Ak, A)

t1 = time.time()

print("Pure Python Taylor Series Result:", y_python)
print("Time taken (s):", t1 - t0)


Pure Python Taylor Series Result: [-161.08381740930085, 199.44510662207463, -148.59264646586863, 153.0297282081668, -174.84320588238367, -203.1500255544525, -5.297887118593034, 117.49692625397924, 299.2846511952939, 69.22458634062573, -132.84153662269728, -37.933471759541845, -29.038251704180603, 332.8779088131362, 242.9308531066957, 141.0616176092088, -66.95674590759563, 210.2468460531768, -226.98158550348825, 134.40517158931013, 3.1523338634030704, -24.49685744926062, -153.4860119379749, -131.88641064319023, -47.64996965150261, -303.02942655920583, 6.946412806998993, 192.70863922170244, 199.27910831194183, 191.3694635037526, 122.30379250579719, -164.23447061439893, 71.10561858491812, -131.47078108963836, -253.82611202495826, 98.80364977113501, -19.786679650648388, 103.06591896672062, 109.64170821912211, -50.748147762898824, 58.94658315698067, -70.0200099047881, -370.48733607024025, -77.77013287504661, 232.97629863978548, 150.10361733246242, 56.28101976051616, 223.28272078320248, -23.

## 2️⃣ NumPy Implementation (Taylor Series)

Compute y = exp(A)·v using **NumPy arrays** for vectorized operations (much faster).

In [19]:
A_np = np.array(A)
v_np = np.array(v)

t0 = time.time()

# Initialize
y_numpy_taylor = v_np.copy()
Ak = A_np.copy()
fact = 1.0

# Taylor series loop
K = 20
for k in range(1, K + 1):
    fact *= k
    y_numpy_taylor += (Ak @ v_np) / fact
    Ak = Ak @ A_np

t1 = time.time()

# Output
print("NumPy Taylor Series Result:", y_numpy_taylor)
print("Time taken (s):", t1 - t0)


NumPy Taylor Series Result: [-2.85860467e+02  3.49533991e+02  1.33876104e+02 -1.80030481e+02
  1.16278663e+02  4.28149880e+01  6.09316433e+02  1.60480752e+02
 -3.94988783e+02 -3.41160014e+02  4.40081341e+02 -2.21244829e+02
 -9.18224068e+01  1.53195144e+02 -2.51760558e+02  4.32048845e+01
 -5.53167396e+01 -1.89141399e+02  2.50614818e+01 -2.64803451e+02
 -7.96305396e+01  5.97145129e+01  1.31088769e+02 -1.46825083e+02
 -1.07418119e+03 -1.57948749e+02 -1.23328850e+02  8.31937416e+01
 -4.93585633e+02 -2.25809840e+02  1.36303825e+01  4.98967388e+01
 -1.97119315e+02 -2.75838566e+02  5.39505138e+02 -2.62018924e+02
 -3.63687452e+02  3.39386001e+02  2.15576103e+02  1.96291748e+02
 -7.73972786e+02  8.80396853e+01 -4.86412333e+02  3.65359981e+02
  5.67897432e+02 -1.21912586e+01 -4.37748535e+01 -4.57396487e+01
  2.17137005e+02 -1.62641623e+02  2.74367322e+01  2.20418125e+02
  2.69178717e+02  3.31083621e+01  1.06489707e+02 -3.95575097e+01
  3.27104387e+02  6.99388389e+02 -3.51468613e+01  6.29350239e+

## 3️⃣ NumPy Implementation (Eigendecomposition)

Compute y = exp(A)·v using **eigendecomposition**:
- A = V D V^-1
- exp(A) = V exp(D) V^-1
- Very fast and accurate.

In [20]:
t0 = time.time()

# Step 1: Eigen decomposition
eigvals, eigvecs = np.linalg.eig(A_np)

# Step 2: Inverse of eigenvector matrix
V = eigvecs
V_inv = np.linalg.inv(eigvecs)

# Step 3: Exponential of diagonal matrix
expD = np.diag(np.exp(eigvals))

# Step 4: Reconstruct exp(A)
expA = V @ expD @ V_inv

# Step 5: Multiply by vector v
y_numpy_eig = expA @ v_np

t1 = time.time()

# Output
print("NumPy Eigendecomposition Result:", y_numpy_eig)
print("Time taken (s):", t1 - t0)


NumPy Eigendecomposition Result: [-2.85863077e+02+3.92011117e-13j  3.49553298e+02-1.00137168e-12j
  1.33862544e+02+5.37991469e-12j -1.80001793e+02+2.06318118e-12j
  1.16279157e+02+3.16889431e-12j  4.28259338e+01+2.41093784e-12j
  6.09337902e+02+3.34232611e-12j  1.60533341e+02-1.55678125e-12j
 -3.94974708e+02+1.76205133e-12j -3.41178756e+02+1.93474423e-12j
  4.40097243e+02+4.53355497e-12j -2.21260834e+02-2.75742384e-12j
 -9.18481789e+01-2.36373406e-12j  1.53190906e+02+3.43797575e-12j
 -2.51781207e+02-2.67495162e-12j  4.31989239e+01+3.45306811e-13j
 -5.53092391e+01-2.75885560e-12j -1.89164439e+02-2.19468580e-13j
  2.50710419e+01-1.71001725e-12j -2.64811580e+02-4.93846391e-12j
 -7.96224637e+01-2.79565339e-12j  5.97384567e+01+3.73295599e-12j
  1.31105721e+02-9.36453081e-13j -1.46806108e+02+3.76749955e-13j
 -1.07424541e+03-4.30612985e-12j -1.57948612e+02-6.42550545e-13j
 -1.23352036e+02+1.08338811e-12j  8.31697692e+01+2.48107712e-12j
 -4.93601342e+02+2.67601215e-12j -2.25807286e+02-3.890725

## Summary
- Pure Python Taylor series: works without NumPy, slow.
- NumPy Taylor series: fast, vectorized.
- Relative difference confirms consistency between methods.